# Overview of Results

Do some analysis on results, assume `kikai` / `not_yeti` have done the hardwork and properly normalised names etc.

## Using Notebook

Ensure prerequisites:

```shell
pip install -r src/visualisations/extra-requirements.txt
```

In [2]:
# Update results etc, remember to update tournaments.yml for what you care about
import os
import subprocess
import sys
from pathlib import Path

env = dict(os.environ)
env["PYTHONUNBUFFERED"] = "1"

proc = subprocess.Popen(
    [sys.executable, "-u", Path("src", "netrunner_results.py")],  # Use same kernel
    cwd=Path("..", ".."),
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
)

try:
    for line in proc.stdout:
        print(line, end="")
finally:
    if proc.stdout:
        proc.stdout.close()
    rc = proc.wait()

[2025-08-02] EMEA Continentals 2025 [https://tournaments.nullsignal.games/tournaments/4110]
could not find identity: None
could not find identity: None
could not find identity: None
could not find identity: None
could not find identity: None
could not find identity: None
could not find identity: None
could not find identity: None
[2025-08-09] APAC Continentals 2025 [https://tournaments.nullsignal.games/tournaments/4118]
nrdb_id not found for Dia [3]
[2025-08-16] APAC Online Continentals 2025 [https://tournaments.nullsignal.games/tournaments/4134]
nrdb_id not found for Lia [5]
[2025-08-23] America Continentals 2025 [https://tournaments.nullsignal.games/tournaments/4129]
nrdb_id not found for sam keilback [14]
nrdb_id not found for Steve [9]
[2025-08-30] EMEA Online Continentals 2025 [https://tournaments.nullsignal.games/tournaments/4149]
could not find identity: None
could not find identity: None


In [1]:
# Data load and utilities
import numpy as np
import pandas as pd
import yaml
from pathlib import Path

faction_colors = {
    "Anarch": "#e21b3c",
    "Criminal": "#0011ff",
    "Shaper": "#2ca02c",
    "HB": "#9467bd",
    "Jinteki": "#e21b3c",
    "Weyland": "#2ca02c",
    "NBN": "#fbff0e",
    "Mini-faction": "#7f7f7f",
    "Neutral Corp": "#7f7f7f",
    "Neutral Runner": "#7f7f7f",
}

LOW_GAME_CUTOFF_P = 0.035

BASE_DIR = Path("..", "..")
DATA_DIR = Path(BASE_DIR, "OUTPUT")
RESULTS_DIR = Path(DATA_DIR, "results")
STANDINGS_DIR = Path(DATA_DIR, "standings")


with open(Path(BASE_DIR, "src", "netrunner", "identities.yml"), "r", encoding="utf-8") as f:
    identities = yaml.safe_load(f)

id_to_faction = {}
for name, data in identities.items():
    faction = data.get("faction")
    id_to_faction[name] = faction
    for alt in data.get("alt_names", []) or []:
        id_to_faction[alt] = faction

valid_identities = set(id_to_faction.keys())


res_frames = []
FILES = sorted(RESULTS_DIR.rglob("*.csv"))
for fp in FILES:
    df = pd.read_csv(
        fp,
        header=0,
        parse_dates=["date"],
        # keep_default_na=False,
        encoding="utf-8",
        engine="python",
    )

    res_frames.append(df)

results = pd.concat(res_frames, ignore_index=True)

# Label byes
mask_invalid = ~results["corp_id"].isin(valid_identities) | ~results["runner_id"].isin(valid_identities)
results.loc[mask_invalid, "result"] = "bye"

# Remove all byes and IDs
mask_byes_ids = results["result"].isin(["bye", "ID"])
results = results[~mask_byes_ids]

standing_frames = []
FILES = sorted(STANDINGS_DIR.rglob("*.csv"))
for fp in FILES:
    df = pd.read_csv(
        fp,
        header=0,
        parse_dates=["date"],
        # keep_default_na=False,
        encoding="utf-8",
        engine="python",
    )

    standing_frames.append(df)

standings = pd.concat(standing_frames, ignore_index=True)

tournaments = results["tournament"].dropna().unique()

print(f"Number of tournaments: {len(FILES)}")
for t in sorted(tournaments):
    print(" -", t)
print(f"Number of games: {len(results)}")
print(f"  Removed {sum(mask_byes_ids)} byes and IDs")

Number of tournaments: 5
 - APAC Continentals 2025
 - APAC Online Continentals 2025
 - America Continentals 2025
 - EMEA Continentals 2025
 - EMEA Online Continentals 2025
Number of games: 1846
  Removed 35 byes and IDs


## Follow Pro Players

Cut data to follow only the best players for some definition of "best"

In [2]:
import re, numpy as np, pandas as pd
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, HoverTool, NumeralTickFormatter, Div
from bokeh.models import RadioButtonGroup, CustomJS
from bokeh.layouts import column
from bokeh.io import output_notebook, output_file, save

output_notebook()


def _apply_low_game_flags(df: pd.DataFrame, cutoff: int) -> pd.DataFrame:
    d = df.copy()
    d["is_low"] = d["games"] < cutoff
    d["alpha"] = np.where(d["is_low"], 0.35, 1.0)  # For fading on graph
    d["winrate_pct"] = (d["winrate"] * 100).round(2)
    d["low_label"] = np.where(d["is_low"], "<b>(LOW)</b>", "")  # Label for tooltip
    return d


def _abbrev_list(items, max_items=7, max_chars=400):
    """Join unique with <br>, trim length, and add '+N more' if overflowing"""
    uniq = list(dict.fromkeys(items))
    shown = uniq[:max_items]
    s = "<br>".join(shown)
    truncated = False
    if len(s) > max_chars:
        s = s[: max_chars - 1].rstrip() + ".."
        truncated = True
    more = max(0, len(uniq) - len(shown))
    if more or truncated:
        s += f"<br><em>+{more} more…</em>" if more else "<br><em>…</em>"
    return s


def _agg_identity_winrate(df: pd.DataFrame, who: str) -> pd.DataFrame:
    d = df.copy()
    d["result"] = d["result"].astype(str).str.lower()

    id_col = f"{who}_id"
    faction_col = f"{who}_faction"
    flag_col = f"{who}_is_cutter"
    player_col = (
        f"{who}_player" if f"{who}_player" in d.columns else ("corp_player" if who == "corp" else "runner_player")
    )

    # Keep only rows where the relevant side was playing
    d = d[d[flag_col]]

    # Add sample label
    d["sample_pair"] = d[player_col].astype(str).str.strip() + " - " + d["tournament"].astype(str)

    grp = d.groupby([id_col, faction_col], dropna=False, as_index=False).agg(
        games=("result", "size"),
        wins=("result", lambda s: (s == who).sum()),
        pairs=("sample_pair", lambda s: _abbrev_list(s)),
    )

    grp["winrate"] = grp["wins"] / grp["games"]
    grp.rename(columns={id_col: "identity", faction_col: "faction", "pairs": "examples"}, inplace=True)
    return grp.sort_values(["winrate", "games"], ascending=[False, False])


def _top_n(df: pd.DataFrame, n: int = 30) -> pd.DataFrame:
    if df.empty:
        return df
    top_by_games = df.sort_values(["games", "winrate"], ascending=[False, False]).head(n).copy()
    top_by_games.sort_values(["winrate", "games", "identity"], ascending=[False, False, True], inplace=True)
    return top_by_games


def _build_flags_for_split(split_key: str) -> pd.DataFrame:
    cutters = standings.copy()
    cutters["top_cut_rank_num"] = pd.to_numeric(cutters["top_cut_rank"], errors="coerce")
    cutters = cutters[cutters["top_cut_rank_num"].notna()]
    cut_player_tournament = cutters[["tournament_id", "name"]].drop_duplicates().rename(columns={"name": "player"})

    if split_key == "tournament-cutter":
        cut_idx = pd.MultiIndex.from_frame(cut_player_tournament)
        corp_match = results.set_index(["tournament_id", "corp_player"]).index.isin(cut_idx)
        runner_match = results.set_index(["tournament_id", "runner_player"]).index.isin(cut_idx)
        mask = corp_match | runner_match
        out = results.loc[mask].copy()
        corp_idx = pd.MultiIndex.from_arrays([out["tournament_id"], out["corp_player"]])
        runner_idx = pd.MultiIndex.from_arrays([out["tournament_id"], out["runner_player"]])
        out["corp_is_cutter"] = corp_idx.isin(cut_idx)
        out["runner_is_cutter"] = runner_idx.isin(cut_idx)
        eligible_names = sorted(cut_player_tournament["player"].unique().tolist())
        return out, eligible_names

    if split_key == "any-cutter":
        cut_idx = cut_player_tournament["player"]
        mask = results["corp_player"].isin(cut_idx) | results["runner_player"].isin(cut_idx)
        out = results.loc[mask].copy()
        out["corp_is_cutter"] = out["corp_player"].isin(cut_idx)
        out["runner_is_cutter"] = out["runner_player"].isin(cut_idx)
        eligible_names = sorted(cut_player_tournament["player"].unique().tolist())
        return out, eligible_names

    m = re.match(r"any-swiss-top(\d+)", split_key)
    if m:
        N = int(m.group(1))
        swiss = standings.copy()
        swiss["swiss_rank_num"] = pd.to_numeric(swiss["swiss_rank"], errors="coerce")
        swiss = swiss[swiss["swiss_rank_num"].notna() & (swiss["swiss_rank_num"] <= N)]
        idx = swiss["name"].drop_duplicates()
        mask = results["corp_player"].isin(idx) | results["runner_player"].isin(idx)
        out = results.loc[mask].copy()
        out["corp_is_cutter"] = out["corp_player"].isin(idx)
        out["runner_is_cutter"] = out["runner_player"].isin(idx)
        eligible_names = sorted(swiss["name"].dropna().unique().tolist())
        return out, eligible_names

    raise ValueError(f"Unknown split_key: {split_key}")


# Make JS payload to hack the bokeh server
def _df_to_payload(df: pd.DataFrame, eligible_names, low_p: float, top_n: int, split_label: str) -> dict:
    low_cut = max(1, int(np.ceil(low_p * len(df)))) if len(df) else 1
    corp_stats = _top_n(_agg_identity_winrate(df, "corp"), top_n)
    runner_stats = _top_n(_agg_identity_winrate(df, "runner"), top_n)

    def _enrich(x: pd.DataFrame) -> pd.DataFrame:
        x = _apply_low_game_flags(x, low_cut)
        x["color"] = x["faction"].map(faction_colors).fillna("#cccccc")
        return x

    corp_stats = _enrich(corp_stats)
    runner_stats = _enrich(runner_stats)

    def _pack(x: pd.DataFrame) -> dict:
        x = x.reset_index(drop=True)
        payload = {}
        for col in [
            "identity",
            "faction",
            "games",
            "wins",
            "winrate",
            "winrate_pct",
            "color",
            "alpha",
            "examples",
            "low_label",
        ]:
            vals = x[col] if col in x.columns else []
            payload[col] = (
                vals.astype(str).tolist()
                if col in ["identity", "faction", "examples", "color", "low_label"]
                else vals.tolist()
            )
        payload["_y_factors"] = x["identity"].astype(str).tolist()
        return payload

    footer_html = (
        f"<h3>Notes!</h3>"
        f"<p>Following {len(eligible_names)} players</p>"
        f"<p>Tournaments ({len(tournaments)}): {', '.join(tournaments)}</p>"
        f"<p>Split type: {split_label}</p>"
        f"<p>Low game cut off (transparent bars): {low_cut} games ({LOW_GAME_CUTOFF_P*100:.1f}% of {len(df)} total games)</p>"
        f"<p>Byes and IDs are excluded.</p>"
    )

    return {"corp": _pack(corp_stats), "runner": _pack(runner_stats), "footer_html": footer_html}


split_keys = [
    "any-cutter",
    "tournament-cutter",
    "any-swiss-top8",
    "any-swiss-top16",
    "any-swiss-top24",
    "any-swiss-top32",
]

# Precompute
TOP_N = 15
payloads = {}
for key in split_keys:
    df_split, eligible_names = _build_flags_for_split(key)
    payloads[key] = _df_to_payload(df_split, eligible_names, LOW_GAME_CUTOFF_P, TOP_N, split_label=key)

# Default = first button
default_key = split_keys[0]
corp_src = ColumnDataSource(payloads[default_key]["corp"])
runner_src = ColumnDataSource(payloads[default_key]["runner"])


# -------------------- plotting bound to sources --------------------
def _make_chart(src: ColumnDataSource, title: str):
    cats = src.data.get("_y_factors", [])
    p = figure(
        height=max(450, 30 * max(3, len(cats))),
        sizing_mode="stretch_width",
        y_range=cats,
        x_range=(0, 1.05),
        title=title,
        toolbar_location="above",
    )
    bars = p.hbar(y="identity", right="winrate", height=0.7, source=src, color="color", alpha="alpha")

    p.add_tools(
        HoverTool(
            renderers=[bars],
            tooltips="""
            <div style="font-size: 12px; line-height: 1.25;">
              <div><b>@identity</b></div>
              <div><b>Faction:</b> @faction</div>
              <div><b>Win rate:</b> @winrate_pct%</div>
              <div><b>Wins/Games:</b> @wins / @games @{low_label}{safe}</div>
              <div style="margin-top:4px;"><b>Examples:</b><br>@examples{safe}</div>
            </div>
            """,
        )
    )
    p.xaxis.axis_label = "Win rate"
    p.xaxis.formatter = NumeralTickFormatter(format="0%")
    p.yaxis.axis_label = ""
    p.x_range.start = 0
    p.outline_line_color = None
    return p


p_corp = _make_chart(corp_src, "Corp")
p_runner = _make_chart(runner_src, "Runner")
footer = Div(text=payloads[default_key]["footer_html"], sizing_mode="stretch_width")

buttons = RadioButtonGroup(labels=split_keys, active=0)

callback = CustomJS(
    args=dict(
        corp_src=corp_src,
        runner_src=runner_src,
        p_corp=p_corp,
        p_runner=p_runner,
        footer=footer,
        data_map=payloads,
        buttons=buttons,
    ),
    code="""
    const key = buttons.labels[buttons.active];

    const corp   = data_map[key]["corp"];
    const runner = data_map[key]["runner"];

    corp_src.data = corp;
    runner_src.data = runner;

    p_corp.y_range.factors = corp["_y_factors"] || [];
    p_runner.y_range.factors = runner["_y_factors"] || [];

    footer.text = data_map[key]["footer_html"];

    corp_src.change.emit();
    runner_src.change.emit();
    """,
)

buttons.js_on_change("active", callback)

layout = column(buttons, p_corp, p_runner, footer, sizing_mode="stretch_width")
show(layout)

Loading BokehJS ...

In [18]:
output_file("cutter_winrates.html", title="Cutter Winrates")
save(layout)

'f:\\personal\\repos\\netrunner_results\\src\\visualisations\\cutter_winrates.html'

In [6]:
# idx = (cutter_results["corp_id"] == "The Zwicky Group: Invisible Hands") & (cutter_results["corp_is_cutter"])
idx = (cutter_results["corp_id"] == "Haas-Bioroid: Precision Design") & (cutter_results["corp_is_cutter"])

cutter_results[idx]

,date,meta,region,location,level,tournament_id,style,tournament,phase,round,table,corp_player,corp_id,corp_faction,result,runner_player,runner_id,runner_faction,corp_is_cutter,runner_is_cutter
50,2025-08-02,25.08,NaN,NaN,NaN,cobra-4110,SSS,EMEA Continentals 2025,swiss,2,1,Pinsel,Haas-Bioroid: Precision Design,HB,runner,Saerleon,"Nyusha ""Sable"" Sintashta: Symphonic Prodigy",Criminal,True,False
59,2025-08-02,25.08,NaN,NaN,NaN,cobra-4110,SSS,EMEA Continentals 2025,swiss,2,10,simen,Haas-Bioroid: Precision Design,HB,corp,giventofly,Sebastião Souza Pessoa: Activist Organizer,Anarch,True,False
102,2025-08-02,25.08,NaN,NaN,NaN,cobra-4110,SSS,EMEA Continentals 2025,swiss,3,1,simen,Haas-Bioroid: Precision Design,HB,corp,Baserton,"Nyusha ""Sable"" Sintashta: Symphonic Prodigy",Criminal,True,False
171,2025-08-02,25.08,NaN,NaN,NaN,cobra-4110,SSS,EMEA Continentals 2025,swiss,4,18,Pinsel,Haas-Bioroid: Precision Design,HB,corp,KingSolomon,"Barry ""Baz"" Wong: Tri-Maf Veteran",Criminal,True,False
209,2025-08-02,25.08,NaN,NaN,NaN,cobra-4110,SSS,EMEA Continentals 2025,swiss,5,4,Pinsel,Haas-Bioroid: Precision Design,HB,corp,"Porkobolo, the roaring pig",Zahya Sadeghi: Versatile Smuggler,Criminal,True,False
260,2025-08-02,25.08,NaN,NaN,NaN,cobra-4110,SSS,EMEA Continentals 2025,swiss,6,3,simen,Haas-Bioroid: Precision Design,HB,corp,MotionBlur,"Nyusha ""Sable"" Sintashta: Symphonic Prodigy",Criminal,True,True
362,2025-08-02,25.08,NaN,NaN,NaN,cobra-4110,SSS,EMEA Continentals 2025,swiss,8,1,simen,Haas-Bioroid: Precision Design,HB,runner,davz131,MuslihaT: Multifarious Marketeer,Criminal,True,True
376,2025-08-02,25.08,NaN,NaN,NaN,cobra-4110,SSS,EMEA Continentals 2025,swiss,8,15,Pinsel,Haas-Bioroid: Precision Design,HB,runner,mallory (l0velace),MuslihaT: Multifarious Marketeer,Criminal,True,True
415,2025-08-02,25.08,NaN,NaN,NaN,cobra-4110,SSS,EMEA Continentals 2025,swiss,9,3,simen,Haas-Bioroid: Precision Design,HB,runner,ZomZraft,Lat: Ethical Freelancer,Shaper,True,True
466,2025-08-02,25.08,NaN,NaN,NaN,cobra-4110,SSS,EMEA Continentals 2025,swiss,10,4,Pinsel,Haas-Bioroid: Precision Design,HB,corp,AceEmpress,Dewi Subrotoputri: Pedagogical Dhalang,Shaper,True,False


In [7]:
standings

,date,meta,region,location,level,tournament_id,style,tournament,top_cut_rank,swiss_rank,...,runner_losses,runner_draws,matchPoints,SoS,xSoS,corp_ID,corp_faction,runner_ID,runner_faction,nrdb_id
0,2025-08-02,25.08,NaN,NaN,NaN,cobra-4110,SSS,EMEA Continentals 2025,5.0,1,...,2,0,26,1.980556,1.772676,Nebula Talent Management: Making Stars,NBN,Hoshiko Shiro: Untold Protagonist,Anarch,35094.0
1,2025-08-02,25.08,NaN,NaN,NaN,cobra-4110,SSS,EMEA Continentals 2025,9.0,2,...,3,0,24,1.800000,1.686451,AU Co.: The Gold Standard in Clones,Jinteki,Lat: Ethical Freelancer,Shaper,40402.0
2,2025-08-02,25.08,NaN,NaN,NaN,cobra-4110,SSS,EMEA Continentals 2025,3.0,3,...,4,0,24,1.580000,1.652329,AU Co.: The Gold Standard in Clones,Jinteki,Zahya Sadeghi: Versatile Smuggler,Criminal,2452.0
3,2025-08-02,25.08,NaN,NaN,NaN,cobra-4110,SSS,EMEA Continentals 2025,1.0,4,...,1,0,23,1.822222,1.762840,AU Co.: The Gold Standard in Clones,Jinteki,MuslihaT: Multifarious Marketeer,Criminal,37519.0
4,2025-08-02,25.08,NaN,NaN,NaN,cobra-4110,SSS,EMEA Continentals 2025,10.0,5,...,1,0,23,1.768571,1.681456,AU Co.: The Gold Standard in Clones,Jinteki,MuslihaT: Multifarious Marketeer,Criminal,17901.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405,2025-08-30,25.08,NaN,online,continental championship,cobra-4149,SSS,EMEA Online Continentals 2025,NaN,97,...,3,0,3,1.117143,1.310270,MirrorMorph: Endless Iteration,HB,"Nyusha ""Sable"" Sintashta: Symphonic Prodigy",Criminal,NaN
406,2025-08-30,25.08,NaN,online,continental championship,cobra-4149,SSS,EMEA Online Continentals 2025,NaN,98,...,2,0,0,2.000000,1.247500,Nebula Talent Management: Making Stars,NBN,"Nyusha ""Sable"" Sintashta: Symphonic Prodigy",Criminal,NaN
407,2025-08-30,25.08,NaN,online,continental championship,cobra-4149,SSS,EMEA Online Continentals 2025,NaN,99,...,1,0,0,1.700000,1.453869,AU Co.: The Gold Standard in Clones,Jinteki,Magdalene Keino-Chemutai: Cryptarchitect,Shaper,21959.0
408,2025-08-30,25.08,NaN,online,continental championship,cobra-4149,SSS,EMEA Online Continentals 2025,NaN,100,...,4,0,0,1.362245,1.235465,Haas-Bioroid: Precision Design,HB,MuslihaT: Multifarious Marketeer,Criminal,31178.0
